># 4 신경망 학습

## 4.1 데이터에서 학습한다!

#### 퍼셉트론 수렴 정리

- 선형 분리 가능 문제는 유한 번의 학습으로 해결 가능함이 증명
- 비선형 분리 문제는 자동으로 학습 불가능

### 4.1.1 데이터 주도 학습

- 알고리즘을 밑바닥부터 설계하는 대신, 주어진 데이터를 활용
- 이미지에서 특징을 추출, 그 특징의 패턴을 기계학습 기술로 학습
- 사람의 개입을 최소화 하지만, 여전히 '사람'이 설계함을 주의

#### 규칙을 배우는 패러다임의 변화

① 사람이 생각한 알고리즘 -> 결과

② 사람이 생각한 특징(SIFT, HOG 등) -> 기계학습(SVM, KNN 등) -> 결과

③ 신경망(딥러닝) -> 결과

※ 종단간 기계학습: 데이터 입력부터 출력까지 사람의 개입이 없는 딥러닝의 특징
- 주어진 데이터 그대로를 입력 데이터로 활용 'end-to-end'로 학습 가능

### 4.1.2 훈련 데이터와 시험 데이터

- 범용 능력(일반화)의 평가를 위해 훈련 데이터와 시험 데이터를 분리

## 4.2 손실 함수

### 4.2.1 평균 제곱 오차

## <center> $E = \frac{1}{2} \sum_{k} (y_{k}-\hat{y}_{k})^2$ </center>

## <center> $E = \frac{1}{2} \sum_{k} (y_{k}-t_{k})^2$ </center>

In [7]:
# 통계학에서 y_hat과 같음
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]

# 통계학에서 y와 같음
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

- t와 같이 한 원소만 1로 나타내는 표기법을 원-핫 인코딩이라고 함

In [8]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y - t) ** 2)

In [13]:
import numpy as np

# 정답은 '2'
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1) '2'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(mean_squared_error(np.array(y), np.array(t)))

# 예2) '7'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(mean_squared_error(np.array(y), np.array(t)))

0.09750000000000003
0.5975


- 제곱 오차가 적은 예1)이 정답에 더 가까움

### 4.2.2 교차 엔트로피 오차

## <center> $E = -\sum_{k}t_{k}log y_{k}$ </center>

- 여기서 $log$는 밑이 $e$인 자연로그($log_{e}$)
- $y_{k}$는 신경망의 출력, $t_{k}$는 정답 레이블
- ex) 신경망 출력이 0.6이라면 교차 엔트로피 오차는 $-log 0.6$ = 0.51

In [14]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

-  아주 작은 값인 delta를 더하는 이유는 no.log() 함수에 0을 입력하면 -inf가 되기 때문

In [15]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1) '2'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

# 예2) '7'일 확률이 가장 높다고 추정
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

0.510825457099338
2.302584092994546


### 4.2.3 미니배치 학습

- 모든 훈련 데이터를 대상으로 손실 함수 값을 구하고 그 총 합을 지표로 삼아야 함
- 하지만 많은 데이터를 대상으로 학습을 하는 것은 현실적으로 무리가 있음
- 훈련 데이터로부터 일부만 골라 학습(표본 추출의 개념)

## <center> $E = -\frac{1}{N}\sum_{n}\sum_{k}t_{nk}log y_{nk}$ </center>

In [16]:
%run mnist.py

import sys, os
sys.path.append(os.pardir)
def load_mnist(normalize=True, flatten=True, one_hot_label=False):

    if not os.path.exists(save_file):
        init_mnist()
        
    with open(save_file, 'rb') as f:
        dataset = pickle.load(f)
    
    # 정규화
    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            dataset[key] /= 255.0
            
    if one_hot_label:
        dataset['train_label'] = _change_one_hot_label(dataset['train_label'])
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])    
    
    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], dataset['test_label']) 

(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [36]:
# 인덱스로 무작위 미니배치를 추출 
train_size = x_train.shape[0]
batch_size = 10
np.random.seed(0)
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

np.random.choice(60000, 10)

array([20757, 55026, 46884, 14935, 15430, 48600, 39512, 52620, 14650,
       17089])

### 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

#### ① 정답 레이블이 원-핫 인코딩일 때

In [39]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y,reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t * np.log(y)) / batch_size

#### ② 정답 레이블이 원-핫 인코딩이 아닐 때

In [40]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y,reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

### 4.2.5 왜 손실 함수를 설정하는가?

- 손실 함수의 값을 가능한 한 작게 하는 매개변수 값을 찾음
- 매개변수의 미분 값을 단서로 값을 서서히 갱신함(과정 반복)
    - ① 미분 값이 양수면 가중치 매개변수를 음의 방향으로 -> 손실 함수의 값을 줄임
    - ② 미분 값이 음수면 가중치 매개변수를 양의 방향으로 -> 손실 함수의 값을 줄임
    - ③ 미분 값이 0이면 가중치 매개변수의 갱신을 멈춤

#### 정확도를 지표로 하면 안되는 이유

- 매개변수 조정을 한다고 해도 불연속적인 변화가 일어남
- 손실 함수를 지표로 사용할 경우에는 연속적인 변화

## 4.3 수치 미분

### 4.3.1 미분

In [ ]:
# 미분 구현